In [1]:
from imbd.data import DataLoader, DataPreprocessor
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import cross_validate, cross_val_predict, GridSearchCV

In [2]:
loader = DataLoader()

In [3]:
df = loader.build_label_20_df()

In [4]:
preprocessor = DataPreprocessor()

In [5]:
train_features = df.drop(loader.labels, axis=1)
train_labels = df[loader.labels]

In [6]:
train_features = preprocessor(train_features)

[Pipeline] ... (step 1 of 4) Processing features_select, total=   0.1s
[Pipeline] ...... (step 2 of 4) Processing quantization, total=   0.1s
[Pipeline] ........... (step 3 of 4) Processing fill_na, total=   0.1s
[Pipeline] . (step 4 of 4) Processing outlier_detection, total=   0.0s


In [7]:
model = MultiOutputRegressor(XGBRegressor())
scorer = make_scorer(mean_squared_error, squared=False)

In [8]:
# res = cross_validate(model, train_features, train_labels, cv=10, scoring=scorer, return_estimator=True)

In [9]:
# res['test_score']

In [ ]:
# best_estimator = res['estimator'][np.argmax(res['test_score'])]
hparams = {
    "estimator__n_estimators": [100, 1000],
    "estimator__max_depth": [2, 5, 10],

}

search = GridSearchCV(model, hparams, cv=10, verbose=2)
search.fit(train_features, train_labels)

In [ ]:
# pred = best_estimator.predict(train_features)
# pred = model.predict(train_features)
# pred = cross_val_predict(model, train_features, train_labels, cv=10)
pred = search.predict(train_features)
pred = np.round(pred, 2)

In [ ]:
res = np.abs(pred - train_labels)
res['mean_abs_diff'] = np.mean(res, axis=1)
res['outlier'] = train_features['outlier']

In [ ]:
res.sort_values('mean_abs_diff', ascending=False).to_csv('output.csv', index=False)

In [ ]:
import plotly.express as px

In [ ]:
columns = train_features.filter(regex='Input_A[0-9]+_*').columns
target = pd.concat([train_features[columns[:24]], train_features['outlier'], res['mean_abs_diff']], axis=1)
target.columns

In [ ]:
px.parallel_coordinates(target, color='mean_abs_diff', color_continuous_scale=px.colors.diverging.Tealrose)

In [ ]:
pred_df = pd.DataFrame(pred, columns=train_labels.columns)

In [ ]:
test = pd.concat([pred_df['Input_A1_020'], train_labels['Input_A1_020']], axis=1)

In [ ]:
test.values